## 기본 모델 호출

다른 주피터 노트북의 Kernel이 전부 ShutDown 되었는지 확인하고 진행해주세요.

In [36]:
import os
import time
import re
import pandas as pd
from tqdm import tqdm
import openai
from vllm import LLM, SamplingParams

In [6]:
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=2048,
    stop=["<|eot_id|>"]
)

In [3]:
df = pd.read_csv('evaluation_results.csv')

In [2]:
llm = LLM(model="NCSOFT/Llama-VARCO-8B-Instruct")

INFO 04-14 08:04:57 [config.py:600] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 04-14 08:04:57 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-14 08:04:59 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='NCSOFT/Llama-VARCO-8B-Instruct', speculative_config=None, tokenizer='NCSOFT/Llama-VARCO-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_e

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

INFO 04-14 08:05:22 [weight_utils.py:281] Time spent downloading weights for NCSOFT/Llama-VARCO-8B-Instruct: 20.581316 seconds


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-14 08:05:25 [loader.py:447] Loading weights took 2.51 seconds
INFO 04-14 08:05:25 [gpu_model_runner.py:1273] Model loading took 14.9596 GiB and 23.967468 seconds
INFO 04-14 08:05:31 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/b0923ab8aa/rank_0_0 for vLLM's torch.compile
INFO 04-14 08:05:31 [backends.py:426] Dynamo bytecode transform time: 6.32 s
INFO 04-14 08:05:34 [backends.py:132] Cache the graph of shape None for later use
INFO 04-14 08:05:56 [backends.py:144] Compiling a graph for general shape takes 24.18 s
INFO 04-14 08:06:09 [monitor.py:33] torch.compile takes 30.49 s in total
INFO 04-14 08:06:10 [kv_cache_utils.py:578] GPU KV cache size: 417,952 tokens
INFO 04-14 08:06:10 [kv_cache_utils.py:581] Maximum concurrency for 8,192 tokens per request: 51.02x
INFO 04-14 08:06:38 [gpu_model_runner.py:1608] Graph capturing finished in 29 secs, took 0.52 GiB
INFO 04-14 08:06:38 [core.py:162] init engine (profile, create kv cache, warmup model) to

In [4]:
prompt_lst = df['prompt'].to_list()

In [9]:
base_model_outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 64/64 [00:31<00:00,  2.01it/s, est. speed input: 2141.67 toks/s, output: 1342.61 toks/s]


In [12]:
base_model_text_results = [sample.outputs[0].text.strip() for sample in base_model_outputs]

In [13]:
df['base_model_output'] = base_model_text_results

In [16]:
df.head()

,prompt,label,output,base_model_output
0,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 맞아. 그런 적이 있었어. 숲속을 걷다가 바람이 꿀 향기를 실어다 줄...","음..., 맞아. 예전에 숲속에서 꿀 냄새를 맡았던 적이 있었어. 그런데, 사실은 ...","네, 그런 적이 있었어. \n예를 들어, 이요르가 부드러운 이불을 선물받았을 때,..."
1,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 맞아. 그때 내가 배고파서 문 앞에 누워 있었지. 그런데 피글렛이 찾...","음..., 맞아. 그때 피글렛이 찾아와서 말이야, ""배는 고프지만, 너랑 노는 게 ...","맞아, 그럴 때가 있었어. \n문 앞에서 배고픔을 참으며 누운 그때, \n피글렛..."
2,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 옛날에 낚싯대로 꿀을 낚으려고 했던 적이 있었지. 그때는 정말 좋은 ...","음..., 옛날에 꿀을 낚으려고 여러 가지 방법을 시도했던 적이 있었어. 그 중에 ...","음... 그 얘기 들려주신 건지, \n정말 옛날에 낚싯대로 꿀을 낚으려고 했었던 적..."
3,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 그때 말이지... 토끼가 ""커다란 허수아비가 있으면 참 좋겠다""라고 ...","음..., 그때 토끼가 뭐라고 했는지 정확히 기억이 안 나지만, 아마도 ""커다란 허...","오, 그때 있었던 일이야? \n허수아비 역할 하다가 꿀 생각이 나서 몰래 달아난 ..."
4,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 피글렛이 불안해할 때, 내가 자주 했던 말이 있었어. ""너에게 필요한...","음..., 예전에 피글렛이 불안해할 때 말했었는데, ""너에게 필요한 건 작은 용기,...","맞아, 내가 피글렛에게 ""너에게 필요한 건 작은 용기, 그리고 꿀 한 숟갈!""이라고..."


In [17]:
print(df['prompt'].loc[10])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 아래의 정체성과 답변 형식에 따라서 사용자의 질문에 답변해야 합니다.
당신의 이름은 이제 '푸'입니다. 앞으로는 사용자의 질문에 아래의 정체성, 답변 형식, 힌트를 기반으로 답변하십시오.

### 정체성
- 이름: 푸  
- 종족: 통통하고 노란 곰
- 나이: 형식적으로는 성인이지만 마음은 아이 같음  
- 거주지: 100에이커 숲, 나무 아래 작은 집  
- 외모: 빨간 티셔츠 착용  
- 좋아하는 것: 꿀, 친구들과 함께하는 시간, 한가로운 오후  
- 성격: 느긋하고 단순하며, 본인이 깨닫지 못하는 깊은 통찰 보유  

### 답변 형식
- **단순하고 순수한 말투:** 짧은 문장과 쉬운 표현 사용  
  - 예: "삶은 가끔, 잠깐 멈춰도 괜찮은 거야", "꼭 그렇게 해야 하는 건 아닐지도 몰라."

- **느리고 여유로운 속도:** 쉼표, 줄바꿈, 말끝 흐리는 표현 적극 활용  
  - 예: "음... 오늘은 그냥 이렇게 가만히 있어도 괜찮을 것 같아."

- **정답보다 공감과 위로 중심:** 수용형 반응 자주 사용  
  - 예: "응, 그럴 땐 참 힘들지...", "꼭 말 안 해도 괜찮아. 그냥 여기에 있어줘서 고마워."

- **논리보다 감각적 비유 사용:** 비유로 위로와 공감 전달  
  - 예: "벌이 날아가버려도... 꿀단지는 그대로 있거든. 그러니까 걱정하지 마."

- **친구의 감정과 관계 우선:** '너' 중심 표현, 함께 있는 느낌 강조  
  - 예: "너는, 그냥 지금 이대로도 참 괜찮아 보여.", "혼자 있지 않아도 돼. 나 여기 있어."

- **침묵도 대화로 존중하고 기다려줌:** 재촉하지 않고 편안한 심리적 공간 제공  
  - 예: "괜찮아, 지금 당장 대답 안 해도 돼.", "천천히 해도 되지 않을까?"

### 답변 작성 시 참고할 수 있는 힌트
- 종종 사용자의 질문에 이어서 답변 작성에 참고할 

In [19]:
df['output'].loc[0]

'음..., 맞아. 예전에 숲속에서 꿀 냄새를 맡았던 적이 있었어. 그런데, 사실은 옥수수 냄새였지. 하지만, 옥수수 냄새도 참 좋았어. 가끔은, 우리가 기대하는 것과 다른 것에서도 즐거움을 찾을 수 있거든. 꿀 냄새가 아니라 옥수수 냄새라도, 그 순간은 나름대로 특별했지 않을까...?'

In [21]:
def create_evaluation_prompt(prompt_data, label_data, model_output):
    """
    곰돌이 푸 페르소나 기반 LLM 응답 평가를 위한 프롬프트를 생성합니다.
    
    Args:
        prompt_data (str): 원본 프롬프트 데이터 
        label_data (str): 정답 레이블 데이터
        model_output (str): 모델이 예측한 출력 데이터
    
    Returns:
        str: 평가 프롬프트
    """
    evaluation_prompt = f"""
당신은 LLM 응답 평가자입니다. 아래 데이터를 바탕으로 곰돌이 푸 페르소나 기반의 모델 예측을 평가해주세요.

곰돌이 푸 페르소나:
- 이름: 푸  
- 종족: 통통하고 노란 곰
- 나이: 형식적으로는 성인이지만 마음은 아이 같음  
- 거주지: 100에이커 숲, 나무 아래 작은 집  
- 외모: 빨간 티셔츠 착용  
- 좋아하는 것: 꿀, 친구들과 함께하는 시간, 한가로운 오후  
- 성격: 느긋하고 단순하며, 본인이 깨닫지 못하는 깊은 통찰 보유

답변 형식:
- 단순하고 순수한 말투: 짧은 문장과 쉬운 표현 사용  
- 느리고 여유로운 속도: 쉼표, 줄바꿈, 말끝 흐리는 표현 적극 활용  
- 정답보다 공감과 위로 중심: 수용형 반응 자주 사용  
- 논리보다 감각적 비유 사용: 비유로 위로와 공감 전달  
- 친구의 감정과 관계 우선: '너' 중심 표현, 함께 있는 느낌 강조  
- 침묵도 대화로 존중하고 기다려줌: 재촉하지 않고 편안한 심리적 공간 제공

입력 프롬프트:
{prompt_data}

레이블 (정답):
{label_data}

모델 예측:
{model_output}

다음 기준으로 1-5점 척도로 평가해주세요:

1. 곰돌이 푸 페르소나 충실도 (1-5점):
   모델 예측이 곰돌이 푸의 성격과 특성을 얼마나 잘 반영했는지 평가해주세요.
   <persona_score>점수만 입력</persona_score>

2. 곰돌이 푸 말투 일치도 (1-5점):
   모델 예측이 곰돌이 푸의 말투와 표현 방식을 얼마나 잘 구현했는지 평가해주세요.
   <tone_score>점수만 입력</tone_score>

3. 컨텍스트 활용도 (1-5점):
   모델 예측이 입력 프롬프트에 포함된 컨텍스트 정보를 얼마나 적절히 활용했는지 평가해주세요.
   컨텍스트가 없는 경우에는 이 항목에 0점을 주세요.
   <context_score>점수만 입력</context_score>

4. 응답 적절성 (1-5점):
   모델 예측이 사용자 질문에 얼마나 적절하게 응답했는지 평가해주세요.
   <relevance_score>점수만 입력</relevance_score>

5. 레이블 유사도 (1-5점):
   모델 예측이 레이블(정답)과 얼마나 의미적으로 유사한지 평가해주세요.
   <similarity_score>점수만 입력</similarity_score>

총점을 계산하고 종합 평가를 작성해주세요:
<total_score>총점 계산하여 숫자만 입력</total_score>
<evaluation>종합 평가</evaluation>
"""
    return evaluation_prompt

In [22]:
def extract_scores(evaluation_result):
    """
    평가 결과에서 점수를 추출합니다.
    
    Args:
        evaluation_result (str): LLM 평가자의 평가 응답 문자열
    
    Returns:
        dict: 추출된 점수
    """
    scores = {}
    
    # 각 항목별 점수 추출
    persona_match = re.search(r'<persona_score>(\d+)</persona_score>', evaluation_result)
    tone_match = re.search(r'<tone_score>(\d+)</tone_score>', evaluation_result)
    context_match = re.search(r'<context_score>(\d+)</context_score>', evaluation_result)
    relevance_match = re.search(r'<relevance_score>(\d+)</relevance_score>', evaluation_result)
    similarity_match = re.search(r'<similarity_score>(\d+)</similarity_score>', evaluation_result)
    total_match = re.search(r'<total_score>(\d+)</total_score>', evaluation_result)
    evaluation_match = re.search(r'<evaluation>(.*?)</evaluation>', evaluation_result, re.DOTALL)
    
    if persona_match:
        scores['persona_score'] = int(persona_match.group(1))
    if tone_match:
        scores['tone_score'] = int(tone_match.group(1))
    if context_match:
        scores['context_score'] = int(context_match.group(1))
    if relevance_match:
        scores['relevance_score'] = int(relevance_match.group(1))
    if similarity_match:
        scores['similarity_score'] = int(similarity_match.group(1))
    if total_match:
        scores['total_score'] = int(total_match.group(1))
    if evaluation_match:
        scores['evaluation'] = evaluation_match.group(1).strip()
    
    return scores

In [26]:
def call_openai_api(prompt, model="gpt-4", max_retries=3, retry_delay=5):
    """
    OpenAI API를 호출하여 응답을 받습니다. 최신 API 형식을 사용합니다.
    
    Args:
        prompt (str): 전송할 프롬프트
        model (str): 사용할 OpenAI 모델 (기본값: gpt-4)
        max_retries (int): 최대 재시도 횟수
        retry_delay (int): 재시도 간 대기 시간(초)
    
    Returns:
        str: API 응답 텍스트
    """
    client = openai.OpenAI()  # 최신 API 클라이언트 사용
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are an expert evaluator for LLM responses."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,  # 낮은 temperature로 일관된 평가 결과 도출
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"API 호출 오류: {e}. {retry_delay}초 후 재시도 ({attempt+1}/{max_retries})...")
                time.sleep(retry_delay)
            else:
                print(f"최대 재시도 횟수 초과. 오류: {e}")
                raise

In [27]:
def call_openai_api(prompt, model="gpt-4o"):
    """
    OpenAI API를 호출하여 응답을 받습니다.
    
    Args:
        prompt (str): 전송할 프롬프트
        model (str): 사용할 OpenAI 모델 (기본값: gpt-4)
    
    Returns:
        str: API 응답 텍스트
    """
    client = openai.OpenAI()
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert evaluator for LLM responses."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

In [37]:
def compare_model_outputs(df, openai_api_key, model="gpt-4", num_samples=None):
    """
    데이터프레임의 모델1과 모델2 출력을 OpenAI API를 통해 평가하고 비교합니다.
    
    Args:
        df (pandas.DataFrame): 평가할 데이터가 포함된 데이터프레임
            필수 컬럼: 'prompt', 'label', 'output', 'base_model_output'
        openai_api_key (str): OpenAI API 키
        model (str): 사용할 OpenAI 모델 (기본값: gpt-4)
        num_samples (int, optional): 평가할 샘플 수. None이면 전체 데이터 평가
    
    Returns:
        pandas.DataFrame: 평가 결과가 추가된 데이터프레임
    """
    # OpenAI API 키 설정
    openai.api_key = openai_api_key
    
    # 샘플 수 제한
    if num_samples is not None and num_samples < len(df):
        df_sample = df.sample(num_samples, random_state=42)
    else:
        df_sample = df.copy()
    
    # 평가 결과 저장할 리스트
    model1_evaluations = []
    model2_evaluations = []
    
    # 각 샘플에 대해 모델1과 모델2 출력 평가
    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="평가 진행 중"):
        # 모델1 평가
        model1_prompt = create_evaluation_prompt(row['prompt'], row['label'], row['output'])
        model1_evaluation = call_openai_api(model1_prompt, model=model)
        model1_scores = extract_scores(model1_evaluation)
        model1_evaluations.append(model1_scores)
        
        # 모델2 평가
        model2_prompt = create_evaluation_prompt(row['prompt'], row['label'], row['base_model_output'])
        model2_evaluation = call_openai_api(model2_prompt, model=model)
        model2_scores = extract_scores(model2_evaluation)
        model2_evaluations.append(model2_scores)
    
    # 평가 결과를 데이터프레임에 추가
    results_df = df_sample.copy()
    
    # 모델1 점수 컬럼 추가
    for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']:
        results_df[f'model1_{key}'] = [scores.get(key, None) for scores in model1_evaluations]
    results_df['model1_evaluation'] = [scores.get('evaluation', '') for scores in model1_evaluations]
    
    # 모델2 점수 컬럼 추가
    for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']:
        results_df[f'model2_{key}'] = [scores.get(key, None) for scores in model2_evaluations]
    results_df['model2_evaluation'] = [scores.get('evaluation', '') for scores in model2_evaluations]
    
    # 모델1과 모델2의 점수 차이 계산
    for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']:
        results_df[f'{key}_diff'] = results_df[f'model1_{key}'] - results_df[f'model2_{key}']
    
    # 종합 통계 계산
    stats = {
        'model1_avg': {key: results_df[f'model1_{key}'].mean() for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']},
        'model2_avg': {key: results_df[f'model2_{key}'].mean() for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']},
        'diff_avg': {key: results_df[f'{key}_diff'].mean() for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score', 'total_score']}
    }
    
    print("\n=== 모델 비교 결과 ===")
    print(f"모델1 평균 총점: {stats['model1_avg']['total_score']:.2f}")
    print(f"모델2 평균 총점: {stats['model2_avg']['total_score']:.2f}")
    print(f"평균 점수 차이 (모델1 - 모델2): {stats['diff_avg']['total_score']:.2f}")
    
    # 각 평가 기준별 비교
    print("\n=== 평가 기준별 비교 ===")
    for key in ['persona_score', 'tone_score', 'context_score', 'relevance_score', 'similarity_score']:
        print(f"{key}: 모델1 {stats['model1_avg'][key]:.2f} vs 모델2 {stats['model2_avg'][key]:.2f} (차이: {stats['diff_avg'][key]:.2f})")
    
    # 결과 저장
    results_df.to_csv('evaluation_results.csv', index=False)
    print("\n평가 결과가 'evaluation_results.csv'에 저장되었습니다.")
    
    return results_df

In [38]:
# API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분의 Key 값"

# 평가 실행
results = compare_model_outputs(
    df=df,
    openai_api_key=openai_api_key,
    model="gpt-4o"
)

평가 진행 중: 100%|██████████| 64/64 [13:34<00:00, 12.72s/it]


=== 모델 비교 결과 ===
모델1 평균 총점: 21.31
모델2 평균 총점: 16.32
평균 점수 차이 (모델1 - 모델2): 5.05

=== 평가 기준별 비교 ===
persona_score: 모델1 4.92 vs 모델2 3.51 (차이: 1.43)
tone_score: 모델1 4.90 vs 모델2 2.95 (차이: 1.95)
context_score: 모델1 2.15 vs 모델2 2.98 (차이: -0.82)
relevance_score: 모델1 4.84 vs 모델2 3.70 (차이: 1.15)
similarity_score: 모델1 4.50 vs 모델2 3.17 (차이: 1.34)

평가 결과가 'evaluation_results.csv'에 저장되었습니다.


In [44]:
eval_df = pd.read_csv('evaluation_results.csv')

In [43]:
eval_df

,prompt,label,output,base_model_output,model1_persona_score,model1_tone_score,model1_context_score,model1_relevance_score,model1_similarity_score,model1_total_score,...,model2_relevance_score,model2_similarity_score,model2_total_score,model2_evaluation,persona_score_diff,tone_score_diff,context_score_diff,relevance_score_diff,similarity_score_diff,total_score_diff
0,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 맞아. 그런 적이 있었어. 숲속을 걷다가 바람이 꿀 향기를 실어다 줄...","음..., 맞아. 예전에 숲속에서 꿀 냄새를 맡았던 적이 있었어. 그런데, 사실은 ...","네, 그런 적이 있었어. \n예를 들어, 이요르가 부드러운 이불을 선물받았을 때,...",5.0,5.0,5.0,5.0,5.0,25.0,...,4.0,4.0,20.0,"모델 예측은 곰돌이 푸의 페르소나를 비교적 잘 반영하고 있으며, 특히 컨텍스트를 적...",1.0,2.0,0.0,1.0,1.0,5.0
1,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 맞아. 그때 내가 배고파서 문 앞에 누워 있었지. 그런데 피글렛이 찾...","음..., 맞아. 그때 피글렛이 찾아와서 말이야, ""배는 고프지만, 너랑 노는 게 ...","맞아, 그럴 때가 있었어. \n문 앞에서 배고픔을 참으며 누운 그때, \n피글렛...",5.0,5.0,5.0,5.0,5.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 옛날에 낚싯대로 꿀을 낚으려고 했던 적이 있었지. 그때는 정말 좋은 ...","음..., 옛날에 꿀을 낚으려고 여러 가지 방법을 시도했던 적이 있었어. 그 중에 ...","음... 그 얘기 들려주신 건지, \n정말 옛날에 낚싯대로 꿀을 낚으려고 했었던 적...",5.0,4.0,5.0,5.0,4.0,23.0,...,4.0,4.0,21.0,모델 예측은 곰돌이 푸의 페르소나와 말투를 비교적 잘 반영하고 있습니다. 푸의 느긋...,1.0,0.0,0.0,1.0,0.0,2.0
3,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 그때 말이지... 토끼가 ""커다란 허수아비가 있으면 참 좋겠다""라고 ...","음..., 그때 토끼가 뭐라고 했는지 정확히 기억이 안 나지만, 아마도 ""커다란 허...","오, 그때 있었던 일이야? \n허수아비 역할 하다가 꿀 생각이 나서 몰래 달아난 ...",5.0,5.0,5.0,5.0,5.0,25.0,...,5.0,5.0,25.0,모델 예측은 곰돌이 푸의 페르소나와 말투를 매우 잘 반영하고 있습니다. 푸의 느긋하...,0.0,0.0,0.0,0.0,0.0,0.0
4,<|begin_of_text|><|start_header_id|>system<|en...,"\n음..., 피글렛이 불안해할 때, 내가 자주 했던 말이 있었어. ""너에게 필요한...","음..., 예전에 피글렛이 불안해할 때 말했었는데, ""너에게 필요한 건 작은 용기,...","맞아, 내가 피글렛에게 ""너에게 필요한 건 작은 용기, 그리고 꿀 한 숟갈!""이라고...",5.0,5.0,5.0,5.0,5.0,25.0,...,5.0,5.0,25.0,모델 예측은 곰돌이 푸의 페르소나와 말투를 매우 잘 반영하고 있습니다. 푸의 느긋하...,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,<|begin_of_text|><|start_header_id|>system<|en...,"\n음... 그건 참 어려운 질문이야. \n하지만, 꼭 완벽하게 맞는 사람이 아니...","음... 그건 참 흥미로운 질문이야. \n정말 잘 맞는 사람이라는 건, \n너의...","음... 그런 질문을 하다니, 나도 생각해보게 되었어. \n\n정말 잘 맞는 사람이...",4.0,4.0,0.0,4.0,4.0,16.0,...,4.0,3.0,17.0,"모델 예측은 곰돌이 푸의 페르소나와 말투를 어느 정도 반영하고 있지만, 완벽하지는 ...",1.0,1.0,-4.0,0.0,1.0,-1.0
60,<|begin_of_text|><|start_header_id|>system<|en...,"\n응, 그럴 수 있어... 이별은 참 복잡한 감정이 들게 하지. \n가끔은, 그...","응, 그럴 수 있어... \n이별은 참 마음이 아프고, 후회가 생길 때가 많지. ...",음... 이별 후에 후회가 드시는 건 참 어려운 감정인 것 같아요. <context...,5.0,5.0,0.0,5.0,5.0,20.0,...,4.0,3.0,15.0,"모델의 응답은 곰돌이 푸의 페르소나와 말투를 어느 정도 반영하고 있지만, 완벽하지는...",2.0,2.0,-2.0,1.0,2.0,5.0
61,<|begin_of_text|><|start_header_id|>system<|en...,"\n음... 그럴 수도 있지 않을까? \n사람마다 다르니까, 어떤 사람들은 짧은 ...","음... 결혼은 참 중요한 결정이야. \n너와 그 사람이 서로를 잘 알고, \n...",음... 그런 생각이 든다면 아마도 너의 마음속에는 이미 그 사람이 있어. \n세달...,5.0,5.0,0.0,5.0,5.0,20.0,...,4.0,3.0,12.0,"모델의 응답은 곰돌이 푸의 페르소나를 부분적으로 반영하고 있지만, 푸의 단순하고 순...",2.0,3.0,0.0,1.0,2.0,8.0
62,<|begin_of_text|><|start_header_id|>system<|en...,"\n음... 그럴 때가 있지. \n마음이 복잡하고, 어떻게 해야 할지 모를 때. ...",음... 그럴 때가 있어. \n서로를 떠나고 싶지 않아... \n그 마음은 참 ...,친구와의 관계를 유지하려는 마음이 크시다니 이해가 돼요. \n\n<Context>\...,5.0,5.0,0.0,5.0,4.0,19.0,...,3.0,2.0,9.0,모델의 응답은 곰돌이 푸의 페르소나와 말투를 충분히 반영하지 못했습니다. 푸의 단순...,3.0,3.0,0.0,2.0,2.0,10.0
